d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# Databricks Delta Time Machine & Optimization Lab

Databricks&reg; Delta allows you to read, write and query data in data lakes in an efficient manner.

## Learning Objectives
In this lab, you will:
* Compare different versions of a Delta table using Time Machine
* Optimize your Delta Lake to increase speed and reduce number of files
* Describe how VACUUM handles invalid files


## Audience
* Primary Audience: Data Engineers
* Secondary Audience: Data Analysts and Data Scientists

## Prerequisites
* Web browser: current versions of Google Chrome, Firefox, Safari, Microsoft Edge and
Internet Explorer 11 on Windows 7, 8, or 10 (see <a href="https://docs.databricks.com/user-guide/supported-browsers.html#supported-browsers#" target="_blank">Supported Web Browsers</a>)
* Databricks Runtime 4.2 or greater
* Completed courses Spark-SQL, DataFrames or ETL-Part 1 from <a href="https://academy.databricks.com/" target="_blank">Databricks Academy</a>, or have similar knowledge

## Datasets Used
We will use online retail datasets from `/mnt/training/online_retail`

### Getting Started

Run the following cell to configure our "classroom."

In [5]:
%run "./Includes/Classroom-Setup"

Because we'll be calculating some aggregates in this notebook, we'll change our partitions after shuffle from the default `200` to `8` (which is a good number for the 8 node cluster we're currently working on).

In [7]:
%python

sqlContext.setConf("spark.sql.shuffle.partitions", "8")

### Check for our previous Delta Lake tables

This lab relies upon some tables created in previous Delta Lake lessons and labs. 

If you get an error from either of the next two SQL queries, running the solution code for the "Delta-Lake-Lab-1" will build all necessary tables.

In [9]:
%sql

SELECT COUNT(*) FROM customer_counts;
SELECT COUNT(*) FROM customer_data_delta;

count(1)
165544


-sandbox
<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> **The following cell will take several minutes to execute, and is only necessary to run if you got an error in the previous cell.**

In [11]:
%run "./Includes/Delta-Lab-2-Prep"

For convenience later in this lab, the paths to the files defining our existing Delta tables are provided. You can use these paths to load the data into DataFrames, if desired, though this entire lab can be completed using SQL on the existant tables.

In [13]:
DeltaPath = userhome + "/delta/customer-data/"
CustomerCountsPath = userhome + "/delta/customer_counts/"

**Note: This lab depends upon the complete exectuion of the notebook titled "Open-Source-Delta-Lake" and the "Delta-Lake-Basics" lab. If these tables don't exist, go back and run all cells in these notebook.**

### Time Travel
Because Delta Lake is version controlled, you have the option to query past versions of the data. Let's look at the history of our current Delta table.

In [16]:
%sql
DESCRIBE HISTORY customer_data_delta

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics
3,2020-03-03T16:16:27.000+0000,2544893342256751,class+005@databricks.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2616396637998472),0302-145314-ajar951,2,WriteSerializable,true,null
2,2020-03-03T16:15:44.000+0000,2544893342256751,class+005@databricks.com,MERGE,Map(predicate -> ((CAST(class_005_databricks_com_db.customer_data_delta.`InvoiceNo` AS BIGINT) = upsert_data.`InvoiceNo`) AND (class_005_databricks_com_db.customer_data_delta.`StockCode` = upsert_data.`StockCode`))),null,List(2616396637998472),0302-145314-ajar951,1,WriteSerializable,false,null
1,2020-03-03T16:15:37.000+0000,2544893342256751,class+005@databricks.com,WRITE,"Map(mode -> Append, partitionBy -> [""Country""])",null,List(2616396637998472),0302-145314-ajar951,0,WriteSerializable,true,null
0,2020-03-03T16:15:30.000+0000,2544893342256751,class+005@databricks.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [""Country""])",null,List(2616396637998472),0302-145314-ajar951,null,WriteSerializable,false,null


Querying an older version is as easy as adding `VERSION AS OF desired_version`. Let's verify that our table from one version back still exists.

In [18]:
%sql
SELECT COUNT(*)
FROM customer_data_delta
VERSION AS OF 1

count(1)
65535


Using a single file storage system, you now have access to every version of your historical data, ensuring that your data analysts will be able to replicate their reports (and compare aggregate changes over time) and your data scientists will be able to replicate their experiments.

### Check difference between versions

You want to compare how many orders from Sweden were added by your recent UPSERT to your BI table.

Let's start by getting the total sum of our `total_orders` column where our country is Sweden.

In [21]:
# ANSWER
count = spark.sql("SELECT SUM(total_orders) FROM customer_counts WHERE Country='Sweden'").collect()[0][0]

print(count)

2983

Again, we can look at the history of our Delta table here.

In [23]:
%sql
DESCRIBE HISTORY customer_counts

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics
1,2020-03-03T16:16:16.000+0000,2544893342256751,class+005@databricks.com,MERGE,Map(predicate -> ((class_005_databricks_com_db.customer_counts.`Country` = class_005_databricks_com_db.new_customer_counts.`Country`) AND (class_005_databricks_com_db.customer_counts.`CustomerID` = class_005_databricks_com_db.new_customer_counts.`CustomerID`))),null,List(2616396637998472),0302-145314-ajar951,0,WriteSerializable,false,null
0,2020-03-03T16:15:58.000+0000,2544893342256751,class+005@databricks.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [""Country""])",null,List(2616396637998472),0302-145314-ajar951,null,WriteSerializable,false,null


Our original table will be version `0`. Let's write a SQL query to see how many orders we originally had from Sweden.

In [25]:
%sql
SELECT SUM(total_orders)
FROM customer_counts
VERSION AS OF 0
WHERE Country='Sweden'

sum(total_orders)
58


We can combine these two queries and get our difference, which represents our new entries.

In [27]:
%sql
SELECT SUM(total_orders) - (
  SELECT SUM(total_orders)
  FROM customer_counts
  VERSION AS OF 0
  WHERE Country='Sweden') AS new_entries
FROM customer_counts
WHERE Country='Sweden'

new_entries
2925


### OPTIMIZE and ZORDER

Let's apply some of these optimizations to `../delta/customer-data/`.

Our data is partitioned by `Country`.

We want to query the data for `StockCode` equal to `22301`.

We expect this query to be slow because we have to examine ALL OF `../delta/customer-data/` to find the desired `StockCode` and not just in one or two partitions.

First, let's time the above query: you will need to form a DataFrame to pass to `preZorderQuery`.

In [29]:
# ANSWER
%timeit preZorderQuery = spark.sql("SELECT * FROM customer_data_delta WHERE StockCode=22301 ").collect()

530 ms ± 26.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Compact the files and re-order by `StockCode`.

In [31]:
%sql
-- ANSWER
OPTIMIZE customer_data_delta
ZORDER by (StockCode)

path,metrics
null,"List(28, 106, List(1496, 619234, 75094.0, 28, 2102647), List(1459, 113115, 25517.0, 106, 2704831), 0, List(minCubeSize(107374182400), List(0, 0), List(111, 2719967), 0, List(106, 2704831), 0), 1)"


Let's time the above query again: you will need to form a DataFrame to pass to `postZorderQuery`.

In [33]:
# ANSWER
%timeit postZorderQuery = spark.sql("SELECT * FROM customer_data_delta WHERE StockCode=22301").collect()

408 ms ± 61.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

### OPTIMIZE your BI table

Here we'll optimize our `customer_counts` table so that we can quickly query on our `CustomerID` column.

In [35]:
%sql
OPTIMIZE customer_counts
ZORDER by (CustomerID)

path,metrics
null,"List(27, 247, List(720, 15131, 7362.0, 27, 198782), List(652, 2031, 1472.0, 247, 363637), 0, List(minCubeSize(107374182400), List(0, 0), List(253, 367765), 0, List(247, 363637), 0), 1)"


Now we can easily look at which of our customers have made the most orders.

In [37]:
%sql
SELECT CustomerID, SUM(total_orders) AS total
FROM customer_counts
GROUP BY CustomerID
ORDER BY total DESC

CustomerID,total
null,26309
12748,693
17841,474
14606,410
15311,400
14911,336
17850,297
13089,257
18118,250
15039,247


Or we can see examine those customers that operate in the most countries.

In [39]:
%sql
SELECT CustomerID, COUNT(Country) AS num_countries
FROM customer_counts
GROUP BY CustomerID
SORT BY num_countries DESC

CustomerID,num_countries
21529,21
20705,21
21095,21
21440,21
21591,21
21137,21
20453,21
20600,21
21160,21
20586,21


And then look at how many orders a customer made in each of these countries.

In [41]:
%sql
SELECT Country, total_orders
FROM customer_counts
WHERE CustomerID = 20059

Country,total_orders
United Kingdom,8
null,5
France,4
Germany,4
United States,7
Netherlands,2
Australia,1
Mexico,2
Austria,3
Sweden,1


### Using VACUUM to clean up small files

After we run OPTIMIZE, we have a number of uncompacted files that are no longer necessary. Running VACUUM will remove these files for us.

Let's go ahead and VACUUM our `customer_data_delta` table, which points at the files in our `DeltaPath` variable.

Count number of files before `VACUUM` for `Country=Sweden`.

In [43]:
# ANSWER
preNumFiles = len(dbutils.fs.ls(DeltaPath + "/Country=Sweden"))
print(preNumFiles)

5

If you try to perform an immediate `VACUUM` (using `RETAIN 0 HOURS` to clean up recently optimized files), you will get an error.

In [45]:
%sql
-- ANSWER
VACUUM customer_data_delta RETAIN 0 HOURS;

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: java.lang.IllegalArgumentException: requirement failed: Are you sure you would like to vacuum files with such a low retention period? If you have
writers that are currently writing to this table, there is a risk that you may corrupt the
state of your Delta table.

If you are certain that there are no operations being performed on this table, such as
insert/upsert/delete/optimize, then you may turn off this check by setting:
spark.databricks.delta.retentionDurationCheck.enabled = false

If you are not sure, please use a value not less than "168 hours".
 
	at scala.Predef$.require(Predef.scala:224)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.checkRetentionPeriodSafety(VacuumCommand.scala:68)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$$anonfun$gc$1$$anonfun$apply$1.apply(VacuumCommand.scala:117)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$$anonfun$gc$1$$anonfun$apply$1.apply(VacuumCommand.scala:103)
	at com.databricks.logging.UsageLogging$$anonfun$recordOperation$1.apply(UsageLogging.scala:428)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.spark.util.PublicDBLogging.withAttributionContext(DatabricksSparkUsageLogger.scala:18)
	at com.databricks.logging.UsageLogging$class.withAttributionTags(UsageLogging.scala:275)
	at com.databricks.spark.util.PublicDBLogging.withAttributionTags(DatabricksSparkUsageLogger.scala:18)
	at com.databricks.logging.UsageLogging$class.recordOperation(UsageLogging.scala:409)
	at com.databricks.spark.util.PublicDBLogging.recordOperation(DatabricksSparkUsageLogger.scala:18)
	at com.databricks.spark.util.PublicDBLogging.recordOperation0(DatabricksSparkUsageLogger.scala:55)
	at com.databricks.spark.util.DatabricksSparkUsageLogger.recordOperation(DatabricksSparkUsageLogger.scala:98)
	at com.databricks.spark.util.UsageLogger$class.recordOperation(UsageLogger.scala:67)
	at com.databricks.spark.util.DatabricksSparkUsageLogger.recordOperation(DatabricksSparkUsageLogger.scala:67)
	at com.databricks.spark.util.UsageLogging$class.recordOperation(UsageLogger.scala:342)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.recordOperation(VacuumCommand.scala:51)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging$class.recordDeltaOperation(DeltaLogging.scala:108)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.recordDeltaOperation(VacuumCommand.scala:51)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$$anonfun$gc$1.apply(VacuumCommand.scala:103)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$$anonfun$gc$1.apply(VacuumCommand.scala:103)
	at com.databricks.sql.acl.CheckPermissions$.trusted(CheckPermissions.scala:806)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.gc(VacuumCommand.scala:102)
	at com.databricks.sql.transaction.directory.VacuumTableCommand$$anonfun$run$1.apply(VacuumTableCommand.scala:58)
	at com.databricks.sql.transaction.directory.VacuumTableCommand$$anonfun$run$1.apply(VacuumTableCommand.scala:43)
	at com.databricks.sql.acl.CheckPermissions$.trusted(CheckPermissions.scala:824)
	at com.databricks.sql.transaction.directory.VacuumTableCommand.run(VacuumTableCommand.scala:43)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:68)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.executeCollect(commands.scala:79)
	at org.apache.spark.sql.Dataset$$anonfun$6.apply(Dataset.scala:206)
	at org.apache.spark.sql.Dataset$$anonfun$6.apply(Dataset.scala:206)
	at org.apache.spark.sql.Dataset$$anonfu

This is a helfpul error. Remember that `VACUUM` is intended for occasional garbage collection. Here we'll just demonstrating that we _can_ use it to clean up files, so we'll set our configuration to allow this operation.

In [47]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", False)

Now we won't get an error when we run `VACUUM`.

In [49]:
%sql
VACUUM customer_data_delta RETAIN 0 HOURS;

Count how many files there are for `Country=Sweden`.

In [51]:
# ANSWER
postNumFiles = len(dbutils.fs.ls(DeltaPath + "/Country=Sweden"))
print(postNumFiles)

Comparing our `preNumFiles` to `postNumFiles`, we can see that this number has reduced.

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>